# Urbansim Buildings Processing

### TODO

Once AWS access is restored:

- Re-run Update_urbansim_buildings_county.ipynb
    - Select distinct -> drop duplicates in urbansim_buildings tables
    
### Verify:

- joinid should be unique to APN

In [1]:
import os
import sys
import pandas as pd


user = os.environ['USER']
sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))
from utils_io import *

## 1. Get data from Socrata

In [3]:
# buildings_2018
socrata_data_id = 'rrrx-2reu'
df = pull_df_from_socrata(socrata_data_id)

pulling data in 37 chunks of 100000 rows each
pulling chunk 0
pulling chunk 1
pulling chunk 2
pulling chunk 3
pulling chunk 4
pulling chunk 5
pulling chunk 6
pulling chunk 7
pulling chunk 8
pulling chunk 9
pulling chunk 10
pulling chunk 11
pulling chunk 12
pulling chunk 13
pulling chunk 14
pulling chunk 15
pulling chunk 16
pulling chunk 17
pulling chunk 18
pulling chunk 19
pulling chunk 20
pulling chunk 21
pulling chunk 22
pulling chunk 23
pulling chunk 24
pulling chunk 25
pulling chunk 26
pulling chunk 27
pulling chunk 28
pulling chunk 29
pulling chunk 30
pulling chunk 31
pulling chunk 32
pulling chunk 33
pulling chunk 34
pulling chunk 35
pulling chunk 36


## 2. Inspect duplicates

In [4]:
# lots of duplicates
print(df.shape)  # (3655207, 15)
df.drop_duplicates(inplace=True)
print(df.shape)  # (3120776, 15)
# Duplicate APNs (makes sense since we're using buildings)
print(df['apn'].nunique())  # 2643041

(3655207, 15)
(3120776, 15)
2643041


## 3. Subset to data with values for EDA

In [5]:
county_cols = ['assessed_building_value', 'assessed_date', 'building_id',
               'building_sqft', 'building_type', 
               'jurisdiction_cty', 'last_sale_date', 'non_residential_sqft',
               'residential_units', 'tenure', 'year_built']

In [6]:
# subset to data with values
df['missing_cty_input'] = df[county_cols].isnull().all(axis=1)
cty_data = df[df['missing_cty_input'] == False].copy()

## 4. EDA

### Check duplicates

It appears that there are duplicate records for buildings where the duplicate record has no building id.

Solution: Drop null values using building_id column

In [ ]:
cty_data.head(10)

Drop null building ids

In [8]:
# # this doesn't work since value is 'nan' not NaN
# cty_data.dropna(subset=['building_id'], inplace=True)

# instead
cty_data = cty_data[cty_data['building_id'] != 'nan']

In [ ]:
cty_data.head(10)

Reassess duplicates

In [10]:
print(cty_data.shape)  # (476300, 16)
# still duplicate building ids
print(cty_data['building_id'].nunique())  # 475110

(476300, 16)
475110


In [19]:
# get target # of buildings
target_num_buildings = cty_data[cty_data['fipco'] == 'CA085']['building_id'].nunique()
target_num_buildings

371818

It looks like the duplicate values are arising from the merge (on APN) of county data and Parcels 2018 data.

Parcels 2018 appears to have APNs for Marin (fipco CA041) and San Francisco (fipco CA075) that match Santa Clara APNs

In [ ]:
vcs = cty_data.groupby('building_id').size().sort_values(ascending=False)
dup_data = cty_data[cty_data['building_id'].isin(vcs[vcs > 1].index)]
dup_data.head()

Now it looks like duplicate values are arising from the same merge, but this time the problem is a many to one joinid:apn relationship.

This is only the case for one APN: 13241104

Solution: select one of the joinids as the correct one (verify with Parcels 2018 which one ought to be chosen)

In [ ]:
print(dup_data['fipco'].value_counts())
sc_dups = dup_data[dup_data['fipco'] == 'CA085']

vcs1 = sc_dups.groupby('building_id').size().sort_values(ascending=False)
dup_sc = sc_dups[sc_dups['building_id'].isin(vcs1[vcs1 > 1].index)]
print(dup_sc['apn'].unique())
dup_sc

### Create cleaned version of data

In [16]:
non_dup = cty_data[cty_data['building_id'].isin(vcs[vcs == 1].index)]
sc_nondup = sc_dups[sc_dups['building_id'].isin(vcs1[vcs1 == 1].index)]

buildings = pd.concat([non_dup, sc_nondup, dup_sc.iloc[[0]]])
buildings = buildings[buildings['fipco'] == 'CA085']

In [20]:
buildings = buildings[buildings['fipco'] == 'CA085']
# building id is now unique
assert len(buildings) == target_num_buildings

### Check match with Parcels 2018 data

In [21]:
# without match: AL, MI, LI, RE, CO, SM, MO, WA, HO

d = {'SJ': 'San Jose',
     'SC': 'Santa Clara',
     'SU': 'Sunnyvale',
     'PA': 'Palo Alto',
     'MV': 'Mountain View',
     'ST': None,
     'LA': 'Los Altos',
     'AL': None,
     'LH': 'Los Altos Hills',
     'PV': 'Portola Valley',
     'MI': None,
     'CA': 'Campbell',
     'CU': 'Cupertino',
     'LG': 'Los Gatos',
     'SA': 'Saratoga',
     'MS': 'Monte Sereno',
     'LI': None,
     'MH': 'Morgan Hill',
     'RE': None,
     'GI': 'Gilroy',
     'CO': None,
     'SM': None,
     'MO': None,
     'WA': None,
     'HO': None}

In [22]:
missing_jurisdict = list(set(buildings['jurisdict'].unique()).difference(set(d.values())))
missing_jurisdict

['Unincorporated Santa Clara']

In [24]:
for j in missing_jurisdict:
    print(j, '\n')
    print(buildings[buildings['jurisdict'] == j]['jurisdiction_cty'].value_counts())
    print('\n')

Unincorporated Santa Clara 

SJ     9813
LG     3158
MH     1870
GI     1641
SM     1408
LA     1245
ST      551
SA      209
nan     160
PA      134
LH      127
CU      117
PV       26
WA       15
MS       14
RE        6
CO        5
SC        5
MV        1
Name: jurisdiction_cty, dtype: int64




In [25]:
buildings['juris_compare'] = buildings['jurisdiction_cty'] + ' ' + buildings['jurisdict']
buildings['juris_compare'].value_counts()

SJ San Jose                       191019
SU Sunnyvale                       27786
SC Santa Clara                     21927
MV Mountain View                   15032
CU Cupertino                       13851
PA Palo Alto                       13734
GI Gilroy                          12483
MH Morgan Hill                     11114
CA Campbell                        10541
SJ Unincorporated Santa Clara       9813
SA Saratoga                         9725
LA Los Altos                        9378
LG Los Gatos                        9202
LG Unincorporated Santa Clara       3158
LH Los Altos Hills                  2793
MH Unincorporated Santa Clara       1870
GI Unincorporated Santa Clara       1641
SM Unincorporated Santa Clara       1408
LA Unincorporated Santa Clara       1245
MS Monte Sereno                     1132
nan San Jose                         605
ST Unincorporated Santa Clara        551
SA Unincorporated Santa Clara        209
nan Unincorporated Santa Clara       160
PA Unincorporate

In [26]:
for k, v in d.items():
    print('\n', v)
    print(buildings[buildings['jurisdiction_cty'] == k]['jurisdict'].value_counts(dropna=False))


 San Jose
San Jose                      191019
Unincorporated Santa Clara      9813
Campbell                          17
Los Gatos                          7
Santa Clara                        2
Saratoga                           1
Name: jurisdict, dtype: int64

 Santa Clara
Santa Clara                   21927
Cupertino                        15
San Jose                         12
Unincorporated Santa Clara        5
Sunnyvale                         5
Name: jurisdict, dtype: int64

 Sunnyvale
Sunnyvale        27786
Mountain View       14
Santa Clara         13
Cupertino            9
San Jose             1
Name: jurisdict, dtype: int64

 Palo Alto
Palo Alto                     13734
Unincorporated Santa Clara      134
Mountain View                    11
Los Altos Hills                  10
Name: jurisdict, dtype: int64

 Mountain View
Mountain View                 15032
Los Altos                        12
Palo Alto                         9
Unincorporated Santa Clara        1
Name: juri

In [27]:
buildings.to_csv('urbanim_buildings_proc.csv', index=False)